In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/international-cp/parsing_categories.csv
/kaggle/input/international-cp/submission.csv
/kaggle/input/international-cp/data/new_train_features.pkl
/kaggle/input/international-cp/data/train_dataset_hackaton2023_train.gzip
/kaggle/input/international-cp/data/hackaton2023_test.gzip
/kaggle/input/international-cp/data/new_test_features.pkl
/kaggle/input/international-cp/grouped_train/grouped_train.pkl


In [3]:
df = pd.read_parquet('/kaggle/input/international-cp/data/train_dataset_hackaton2023_train.gzip')

cats = pd.read_csv('/kaggle/input/international-cp/parsing_categories.csv')

In [5]:
df = df.merge(cats[['dish_name', 'item_category', 'item_price']], on='dish_name', how='left')

In [6]:
from sklearn.preprocessing import LabelEncoder

le_format = LabelEncoder()

df['format_name'] = le_format.fit_transform(df['format_name'])

le_dish = LabelEncoder()

df['item_category'] = le_dish.fit_transform(df['item_category'])

In [7]:
def time_of_day(hour):
    if hour < 6:
        return 'night'
    elif hour < 12:
        return 'morning'
    elif hour < 17:
        return 'noon'
    else:
        return 'evening'

In [8]:
def is_weekend(day):
    if day in [5, 6]:
        return 1
    else:
        return 0

In [9]:
df['day_of_week'] = df['startdatetime'].dt.dayofweek
df['month'] = df['startdatetime'].dt.month
df['hour'] = df['startdatetime'].dt.hour
df['time_of_day'] = df['hour'].apply(time_of_day)
df['is_weekend'] = df['day_of_week'].apply(is_weekend)

In [10]:
df['format_name'] = df['format_name'].astype(str)

df['item_category'] = df['item_category'].astype(str)

In [12]:
df['price_revenue_diff'] = df['item_price'] - df['revenue']

In [15]:
import pandas as pd
import numpy as np

def aggregate_features(df):
    revenue_agg = df.groupby('customer_id')['revenue'].agg(['min', 'max', 'mean', 'median', 'std', 'count', 'sum']).rename(columns=lambda x: f'revenue_{x}')

    area_agg = df.groupby('customer_id')['ownareaall_sqm'].agg(['min', 'max', 'mean', 'median', 'std', 'count']).rename(columns=lambda x: f'ownareaall_sqm_{x}')

    diff_agg = df.groupby('customer_id')['price_revenue_diff'].agg(['min', 'max', 'mean', 'median', 'std', 'count', 'sum']).rename(columns=lambda x: f'price_revenue_diff_{x}')
    
    format_name_freq = df.groupby(['customer_id', 'format_name']).size().unstack(fill_value=0).add_prefix('format_name_').add_suffix('_frequency')
    
    category_freq = df.groupby(['customer_id', 'item_category']).size().unstack(fill_value=0).add_prefix('item_category_').add_suffix('_frequency')

    # Frequency of purchases by day of week
    day_of_week_freq = df.groupby(['customer_id', 'day_of_week']).size().unstack(fill_value=0).add_prefix('dayofweek_').add_suffix('_frequency')
    month_freq = df.groupby(['customer_id', 'month']).size().unstack(fill_value=0).add_prefix('month_').add_suffix('_frequency')
    time_of_day_freq = df.groupby(['customer_id', 'time_of_day']).size().unstack(fill_value=0).add_prefix('time_of_day_').add_suffix('_frequency')
    hour_freq = df.groupby(['customer_id', 'hour']).size().unstack(fill_value=0).add_prefix('hour_').add_suffix('_frequency')
    is_weekend_freq = df.groupby(['customer_id', 'is_weekend']).size().unstack(fill_value=0).add_prefix('is_weekend_').add_suffix('_frequency')
    
    # Calculate average days between purchases
    df.sort_values(['customer_id', 'startdatetime'], inplace=True)
    df['previous_purchase'] = df.groupby('customer_id')['startdatetime'].shift(1)
    df['days_between_purchases'] = (df['startdatetime'] - df['previous_purchase']).dt.days
    days_between = df.groupby('customer_id')['days_between_purchases'].agg(['min', 'max', 'mean', 'median', 'std', 'count']).fillna(-1).rename(columns=lambda x: f'days_between_purchases_{x}')

    df['hours_between_purchases'] = (df['startdatetime'] - df['previous_purchase']).dt.total_seconds() / 3600
    hours_between = df.groupby('customer_id')['hours_between_purchases'].agg(['min', 'max', 'mean', 'median', 'std', 'count']).fillna(-1).rename(columns=lambda x: f'hours_between_purchases_{x}')
    
    # Merging all aggregated features
    final_df = pd.concat([revenue_agg, area_agg, diff_agg, format_name_freq, category_freq, day_of_week_freq, month_freq, time_of_day_freq, hour_freq, is_weekend_freq, days_between, hours_between], axis=1).reset_index()
    return final_df

# Example usage
# aggregated_data = aggregate_features(df)

df_agg = aggregate_features(df)

In [16]:
df.index = df.customer_id

In [17]:
df = df.drop(columns=['customer_id'])

In [18]:
def generate_pivot_features(df, col, value, aggfunc):
    pivot = df.pivot_table(values=[value], index=df.index, columns=[col], aggfunc=aggfunc)
    pivot.columns = [f'{col}_{v}_{value}_{aggfunc}' for _, v in pivot.columns]
#     pivot = pivot.reset_index()

    return pivot

In [19]:
def preprocess(df):
    aggs = ['min', 'max', 'mean', 'median', 'std', 'count', 'sum']
    columns = ['format_name', 'day_of_week', 'month', 'hour', 'time_of_day', 'is_weekend', 'item_category']
    pivot_tables = []
    
    
    for col in columns:
        for agg in aggs:
            col_agg = generate_pivot_features(df, col, 'revenue', agg)
            pivot_tables.append(col_agg)
            
    # the rest
    
    data = pd.concat(pivot_tables, axis=1)

    return data

In [20]:
data = preprocess(df)

In [22]:
target = df.groupby('customer_id').buy_post.first()

In [23]:
df_agg.index = df_agg.customer_id

In [24]:
df_agg = df_agg.drop(columns=['customer_id'])

In [26]:
df_agg = df_agg.join(data)

In [35]:
nan_percentage = df_agg.isna().mean()
cols_to_drop = nan_percentage[nan_percentage > 0.9].index
df_cleaned = df_agg.drop(columns=cols_to_drop)

In [37]:
def get_correlated_feats(corr_matrix, feat_stats, greater_is_better=True, corr_threshold=0.9):
    cols = corr_matrix.columns.to_list()
    dropped = {col:0 for col in cols}
    for col in tqdm.tqdm(cols, desc='Get correlated features'):
        if dropped[col] == 0:
            columns_to_check = corr_matrix.index.values[np.abs(corr_matrix[col]) >= corr_threshold]
            if len(columns_to_check) > 1:
                print(f'{col} -> {columns_to_check}')
                if feat_stats is None:
                    bad_cols = columns_to_check[1:]
                else:
                    sel_stats = feat_stats.loc[columns_to_check]
                    if greater_is_better:
                        bad_cond = np.abs(sel_stats) < np.abs(sel_stats).max()
                    else:
                        bad_cond = np.abs(sel_stats) > np.abs(sel_stats).min()
                        
                    bad_cols = sel_stats[bad_cond].index.to_list()
                    norm_cols = sel_stats[~bad_cond].index.to_list()
                    if len(norm_cols) > 1:
                        for norm_col in norm_cols[1:]:
                            dropped[norm_col] += 1
                            
                for bad_col in bad_cols:
                    dropped[bad_col] += 1
                
    high_corr_cols = [c for c in dropped.keys() if dropped[c] > 0]
    return high_corr_cols

In [38]:
corr_mx = df_cleaned.corr()

In [41]:
import tqdm
corr_feats = get_correlated_feats(corr_mx, feat_stats=nan_percentage, greater_is_better=False, corr_threshold=0.9)

Get correlated features:  47%|████▋     | 215/460 [00:00<00:00, 1085.68it/s]

revenue_max -> ['revenue_max' 'revenue_std']
revenue_count -> ['revenue_count' 'ownareaall_sqm_count' 'price_revenue_diff_count'
 'days_between_purchases_count' 'hours_between_purchases_count'
 'is_weekend_0_revenue_count']
ownareaall_sqm_mean -> ['ownareaall_sqm_mean' 'ownareaall_sqm_median']
format_name_1_frequency -> ['format_name_1_frequency' 'format_name_1_revenue_count']
format_name_2_frequency -> ['format_name_2_frequency' 'format_name_2_revenue_count']
format_name_6_frequency -> ['format_name_6_frequency' 'format_name_6_revenue_count']
format_name_7_frequency -> ['format_name_7_frequency' 'format_name_7_revenue_count']
format_name_8_frequency -> ['format_name_8_frequency' 'format_name_8_revenue_count']
item_category_1_frequency -> ['item_category_1_frequency' 'item_category_1_revenue_count']
item_category_10_frequency -> ['item_category_10_frequency' 'item_category_10_revenue_count']
item_category_2_frequency -> ['item_category_2_frequency' 'item_category_2_revenue_count']
item

Get correlated features: 100%|██████████| 460/460 [00:00<00:00, 1386.96it/s]

month_10_revenue_mean -> ['month_10_revenue_mean' 'month_10_revenue_median']
month_11_revenue_mean -> ['month_11_revenue_mean' 'month_11_revenue_median']
month_12_revenue_mean -> ['month_12_revenue_mean' 'month_12_revenue_median']
hour_10_revenue_max -> ['hour_10_revenue_max' 'hour_10_revenue_std']
hour_11_revenue_max -> ['hour_11_revenue_max' 'hour_11_revenue_std'
 'time_of_day_morning_revenue_max']
hour_12_revenue_max -> ['hour_12_revenue_max' 'hour_12_revenue_std']
hour_13_revenue_max -> ['hour_13_revenue_max' 'hour_13_revenue_std']
hour_14_revenue_max -> ['hour_14_revenue_max' 'hour_14_revenue_std']
hour_15_revenue_max -> ['hour_15_revenue_max' 'hour_15_revenue_std']
hour_16_revenue_max -> ['hour_16_revenue_max' 'hour_16_revenue_std']
hour_17_revenue_max -> ['hour_17_revenue_max' 'hour_17_revenue_std']
hour_18_revenue_max -> ['hour_18_revenue_max' 'hour_18_revenue_std']
hour_19_revenue_max -> ['hour_19_revenue_max' 'hour_19_revenue_std']
hour_20_revenue_max -> ['hour_20_revenue_max

In [42]:
len(corr_feats)

140

In [43]:
df_no_corr = df_cleaned.drop(columns=corr_feats)
df_no_corr.shape

(500000, 320)

In [47]:
float64_cols = list(df_no_corr.select_dtypes(include='float64'))

df_no_corr[float64_cols] = df_no_corr[float64_cols].astype('float32')

In [48]:
df_no_corr.to_pickle('train_agg_320_feats_no_corr_no_target.pkl')

In [56]:
test = pd.read_parquet('/kaggle/input/international-cp/data/hackaton2023_test.gzip')

In [60]:
test = test.merge(cats[['dish_name', 'item_category', 'item_price']], on='dish_name', how='left')

In [61]:
test.head()

,customer_id,group_name,revenue,startdatetime,dish_name,ownareaall_sqm,format_name,item_category,item_price
0,13220760,test,0.28,2022-12-22 13:09:11,"Липтон Лимон в бутылке 0,5л",174.4,Фудкорт с туалетом,Холодные напитки,114.99
1,30315975,test,139.99,2023-03-05 16:45:05,Пиво Балтика 7 0.5л розлив,235.0,Отдельно стоящий,Пиво,NaN
2,21679985,test,36.00,2022-12-13 16:10:55,Кинг Фри мал,165.0,Фудкорт без туалета,Картофель,84.99
3,29754274,test,230.00,2023-03-14 14:10:43,Цезарь ролл со стрипсами,150.0,Фудкорт без туалета,Роллы,249.99
4,7797823,test,204.96,2022-10-22 14:09:28,Up Гранд Чиз Фреш,114.5,Фудкорт без туалета,Говядина,254.99


In [62]:
test['format_name'] = le_format.transform(test.format_name)
test['item_category'] = le_dish.transform(test.item_category)

In [63]:
test['day_of_week'] = test['startdatetime'].dt.dayofweek
test['month'] = test['startdatetime'].dt.month
test['hour'] = test['startdatetime'].dt.hour
test['time_of_day'] = test['hour'].apply(time_of_day)
test['is_weekend'] = test['day_of_week'].apply(is_weekend)

In [68]:
test['format_name'] = test['format_name'].astype(str)
test['item_category'] = test['item_category'].astype(str)

In [69]:
test['price_revenue_diff'] = test['item_price'] - test['revenue']

In [70]:
test_agg = aggregate_features(test)

In [71]:
test.index = test.customer_id
test = test.drop(columns=['customer_id'])

In [72]:
data_test = preprocess(test)

In [73]:
test_agg.index = test_agg.customer_id
test_agg = test_agg.drop(columns=['customer_id'])

In [74]:
test_agg = test_agg.join(data_test)

In [77]:
df_agg = pd.read_pickle('/kaggle/working/train_agg_320_feats_no_corr_no_target.pkl')
test_agg = test_agg[df_agg.columns.to_list()]

In [79]:
float64_cols = list(test_agg.select_dtypes(include='float64'))

test_agg[float64_cols] = test_agg[float64_cols].astype('float32')

/tmp/ipykernel_47/4042967504.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_agg[float64_cols] = test_agg[float64_cols].astype('float32')


In [81]:
test_agg.to_pickle('test_agg_320_feats_no_corr.pkl')